In [1]:
import os

from langchain_anthropic import ChatAnthropic
from llama_index.core import Settings, Document, PropertyGraphIndex 
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.indices.property_graph import VectorContextRetriever, CypherTemplateRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.llms.anthropic import Anthropic

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
LLAMA_API_KEY = os.getenv('LLAMA_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = CLAUDE_API_KEY
os.environ["LLAMA_CLOUD_API_KEY"] = LLAMA_API_KEY

In [3]:
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    stop=["\n\nHuman"],
)

llama_llm = Anthropic(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0
)

llama_openai_embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "15082001"
NEO4J_DATABASE = "neo4j"

graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
    refresh_schema=False,
)

# gds = GraphDataScience(NEO4J_URI, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:
index = PropertyGraphIndex.from_existing(
    llm = llama_llm,
    embed_model=llama_openai_embed_model,
    property_graph_store=graph_store,
)

### Vector Similarity

In [ ]:
sub_retriever = VectorContextRetriever(
  index.property_graph_store, 
  vector_store=index.vector_store,
  embed_model=llama_openai_embed_model,
)

## Cypher Queries

In [ ]:
class Params(BaseModel):
    """Parameters for a cypher query."""
    names: list[str] = Field(description="A list of possible entity names or keywords related to the query.")

In [ ]:
cypher_query = """
   MATCH (c:Chunk)-[:MENTIONS]->(o) 
   WHERE o.name IN $names
   RETURN c.text, o.name, o.label;
"""
   
sub_retriever = CypherTemplateRetriever(
 index.property_graph_store, 
 Params, 
 cypher_query,
 llm=llm,
)

## Get Relationship Paths

In [ ]:
# query nodes
llama_node = graph_store.get(properties={"name": "llama"})[0]

# get relationship paths  
paths = graph_store.get_rel_map([llama_node], depth=1)